In [1]:
import numpy as np
import pandas as pd
import allel
import re

def load_vcf(vcf_path, metadata):
    """
    Load VCF and filter poor-quality samples
    """
    
    sampleIDs = metadata.sampleID.to_list()
    
    # load vcf and get genotypes and positions
    vcf = allel.read_vcf(vcf_path, fields='*')
    samples = vcf['samples']
    # keep only samples in qcpass metadata 
    sample_mask = np.isin(vcf['samples'], metadata.sampleID)
    
    # remove low quality samples 
    geno = allel.GenotypeArray(vcf['calldata/GT'])
    geno = geno.compress(sample_mask, axis=1)
    pos = vcf['variants/POS']
    contig = vcf['variants/CHROM']
    indel = vcf['variants/INDEL']
    
    # remove indels 
    geno = geno.compress(~indel, axis=0)
    pos = pos[~indel]
    contig = contig[~indel]
    
    return geno, pos, contig, samples[sample_mask], vcf['variants/REF'][~indel], vcf['variants/ALT'][~indel]

In [2]:
metadata_path = '../../../results/config/metadata.qcpass.tsv'
kdr_marker_snps_path = '../../../resources/ag-vampir/Kdr_marker_SNPs.csv'
vcf_path = "../../../results/vcfs/targets/ampseq-vigg-01.annot.vcf"
cohort_cols = 'location,taxon'
wkdir = "../../.."

### Load in the table of SNPs that we use to define haplotypes

In [3]:
cohort_cols = cohort_cols.split(",")

hap_def = pd.read_csv(kdr_marker_snps_path, sep = '\t', index_col = 1)
hap_def['variant_pos'] = hap_def.index.str.replace('.*:', '', regex = True).astype('int')
hap_def

,SNP name,Allele1,Allele1 interpretation,Allele2,Allele2 interpretation,variant_pos
SNP Position,,,,,,
2L:2422651,kdr-995S,T,Not KdrS,C,KdrS,2422651
2L:2422652,kdr-995F,A,Not KdrF,T,KdrF,2422652
2L:2391228,kdr-402L,G,Not 402L,C or T,V402L,2391228
2L:2403941,Def-S4S5,T,Not S4S5,C,kdr haplotype cluster S4 or S5,2403941
2L:2414062,Def-F3F4-2,A,Not F3F4,T,kdr haplotype cluster F3 or F4,2414062
2L:2399997,Def-F3,G,Not F3,C,kdr haplotype cluster F3,2399997
2L:2425052,Def-F2,G,Not F2,A,kdr haplotype cluster F2,2425052
2L:2380982,Def-S1-3,T,Not S1,C,kdr haplotype cluster S1,2380982
2L:2408677,Def-S5,A,Not S5,C,kdr haplotype cluster S5,2408677


### The functions for making the kdr background calls

In [4]:
# Determine kdr F origin for a genotype
def _F_kdr_origin_gen(genotypes, clean = True):
    if 'sample_name' in genotypes.index:
        sample_name = genotypes['sample_name']
    else:
        sample_name = genotypes.name
    # Check for the 995F mutations
    if pd.isnull(genotypes['kdr-995F']):
        kdr_F_origins = 'F:unknown'
    elif genotypes['kdr-995F'] == 'AA':
        kdr_F_origins = 'F:wt_hom'
    elif genotypes['kdr-995F'] == 'AT':
        kdr_F_origins = 'F:het'
    elif genotypes['kdr-995F'] == 'TT':
        kdr_F_origins = 'F:hom'
    else:
        print(f'Unexpected kdr F genotype. {sample_name} {genotypes["kdr-995F"]}')
        kdr_F_origins = 'Fail. Unexpected kdr F genotype'
    # If the individual has Fkdr, find out its origins
    # For F homozygotes
    if kdr_F_origins == 'F:hom':
        if pd.isnull(genotypes['Def-F1']):
            kdr_F_origins = f'{kdr_F_origins},F1?'
        elif genotypes['Def-F1'] == 'AA':
            kdr_F_origins = f'{kdr_F_origins},F1_hom'
        elif genotypes['Def-F1'] == 'AG':
            kdr_F_origins = f'{kdr_F_origins},F1_het'
        #
        if pd.isnull(genotypes['Def-F2']):
            kdr_F_origins = f'{kdr_F_origins},F2?'
        elif genotypes['Def-F2'] == 'AA':
            kdr_F_origins = f'{kdr_F_origins},F2_hom'
        elif genotypes['Def-F2'] == 'AG':
            kdr_F_origins = f'{kdr_F_origins},F2_het'
        #
        if pd.isnull(genotypes['Def-F3F4-2']):
            kdr_F_origins = f'{kdr_F_origins},F3F4?'
        elif genotypes['Def-F3F4-2'] == 'TT':
            if pd.isnull(genotypes['Def-F3']):
                kdr_F_origins = f'{kdr_F_origins},(F3F4)_hom'
            elif genotypes['Def-F3'] == 'CC':
                kdr_F_origins = f'{kdr_F_origins},F3_hom'
            elif genotypes['Def-F3'] == 'CG':
                kdr_F_origins = f'{kdr_F_origins},F3_het,F4_het'
            elif genotypes['Def-F3'] == 'GG':
                kdr_F_origins = f'{kdr_F_origins},F4_hom'
        elif genotypes['Def-F3F4-2'] == 'AT':
            if pd.isnull(genotypes['Def-F3']):
                kdr_F_origins = f'{kdr_F_origins},(F3F4)_het'
            elif genotypes['Def-F3'] == 'CC':
                kdr_F_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for F3F4, but homozygote for F3.'
            elif genotypes['Def-F3'] == 'CG':
                kdr_F_origins = f'{kdr_F_origins},F3_het'
            elif genotypes['Def-F3'] == 'GG':
                kdr_F_origins = f'{kdr_F_origins},F4_het'
        #
        if pd.isnull(genotypes['Def-F5-2']):
            kdr_F_origins = f'{kdr_F_origins},F5?'
        elif genotypes['Def-F5-2'] == 'GG':
            kdr_F_origins = f'{kdr_F_origins},F5_hom'
        elif genotypes['Def-F5-2'] == 'AG':
            kdr_F_origins = f'{kdr_F_origins},F5_het'
    # for F heterozygotes
    elif kdr_F_origins == 'F:het':
        if pd.isnull(genotypes['Def-F1']):
            kdr_F_origins = f'{kdr_F_origins},F1?'
        elif genotypes['Def-F1'] == 'AA':
            kdr_F_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for F kdr, but homozygote for F1.'
        elif genotypes['Def-F1'] == 'AG':
            kdr_F_origins = f'{kdr_F_origins},F1_het'
        #
        if pd.isnull(genotypes['Def-F2']):
            kdr_F_origins = f'{kdr_F_origins},F2?'
        elif genotypes['Def-F2'] == 'AA':
            kdr_F_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for F kdr, but homozygote for F2.'
        elif genotypes['Def-F2'] == 'AG':
            kdr_F_origins = f'{kdr_F_origins},F2_het'
        #
        if pd.isnull(genotypes['Def-F3F4-2']):
            kdr_F_origins = f'{kdr_F_origins},F3F4?'
        elif genotypes['Def-F3F4-2'] == 'TT':
            kdr_F_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for F kdr, but homozygote for F3F4.'
        elif genotypes['Def-F3F4-2'] == 'AT':
            if pd.isnull(genotypes['Def-F3']):
                kdr_F_origins = f'{kdr_F_origins},(F3F4)_het'
            elif genotypes['Def-F3'] == 'CC':
                kdr_F_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for F kdr and F3F4, but homozygote for F3.'
            elif genotypes['Def-F3'] == 'CG':
                kdr_F_origins = f'{kdr_F_origins},F3_het'
            elif genotypes['Def-F3'] == 'GG':
                kdr_F_origins = f'{kdr_F_origins},F4_het'
        #
        if pd.isnull(genotypes['Def-F5-2']):
            kdr_F_origins = f'{kdr_F_origins},F5?'
        elif genotypes['Def-F5-2'] == 'GG':
            kdr_F_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for F kdr, but homozygote for F5.'
        elif genotypes['Def-F5-2'] == 'AG':
            kdr_F_origins = f'{kdr_F_origins},F5_het'
    if clean:
        return(_kdr_gen_cleanup(kdr_F_origins))
    else:
        return(kdr_F_origins)


# Determine kdr S origin for a genotype
def _S_kdr_origin_gen(genotypes, clean = True, alternate_S4S5 = False):
    if 'sample_name' in genotypes.index:
        sample_name = genotypes['sample_name']
    else:
        sample_name = genotypes.name
    # Check for the 995S mutations
    if pd.isnull(genotypes['kdr-995S']):
        kdr_S_origins = 'S:unknown'
    elif genotypes['kdr-995S'] == 'TT':
        kdr_S_origins = 'S:wt_hom'
    elif genotypes['kdr-995S'] == 'CT':
        kdr_S_origins = 'S:het'
    elif genotypes['kdr-995S'] == 'CC':
        kdr_S_origins = 'S:hom'
    else:
        print(f'Unexpected kdr S genotype. {sample_name} {genotypes["kdr-995S"]}')
        kdr_S_origins = 'Fail. Unexpected kdr S genotype'
    # If the individual has Skdr, find out its origins
    # For S homozygotes
    if kdr_S_origins == 'S:hom':
        if pd.isnull(genotypes['Def-S1-3']):
            kdr_S_origins = f'{kdr_S_origins},S1?'
        elif genotypes['Def-S1-3'] == 'CC':
            kdr_S_origins = f'{kdr_S_origins},S1_hom'
        elif genotypes['Def-S1-3'] == 'CT':
            kdr_S_origins = f'{kdr_S_origins},S1_het'
        #
        if pd.isnull(genotypes['Def-S2S4']):
            kdr_S_origins = f'{kdr_S_origins},S2S4?'
        elif genotypes['Def-S2S4'] == 'TT':
            if pd.isnull(genotypes['Def-S2-4']):
                kdr_S_origins = f'{kdr_S_origins},(S2S4)_hom'
            elif genotypes['Def-S2-4'] == 'AA':
                kdr_S_origins = f'{kdr_S_origins},S2_hom'
            elif genotypes['Def-S2-4'] == 'AT':
                kdr_S_origins = f'{kdr_S_origins},S2_het,S4_het'
            elif genotypes['Def-S2-4'] == 'TT':
                kdr_S_origins = f'{kdr_S_origins},S4_hom'
        elif genotypes['Def-S2S4'] == 'CT':
            if pd.isnull(genotypes['Def-S2-4']):
                kdr_S_origins = f'{kdr_S_origins},(S2S4)_het'
            elif genotypes['Def-S2-4'] == 'AA':
                kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S2S4, but homozygote for S2.'
            elif genotypes['Def-S2-4'] == 'AT':
                kdr_S_origins = f'{kdr_S_origins},S2_het'
            elif genotypes['Def-S2-4'] == 'TT':
                kdr_S_origins = f'{kdr_S_origins},S4_het'
        #
        if pd.isnull(genotypes['Def-S3']):
            kdr_S_origins = f'{kdr_S_origins},S3?'
        elif genotypes['Def-S3'] == 'GG':
            kdr_S_origins = f'{kdr_S_origins},S3_hom'
        elif genotypes['Def-S3'] == 'GT':
            kdr_S_origins = f'{kdr_S_origins},S3_het'
        # 
        if alternate_S4S5:
            if pd.isnull(genotypes['Def-S4S5-2']):
                kdr_S_origins = f'{kdr_S_origins},S4S5?'
            elif genotypes['Def-S4S5-2'] == 'TT':
                if pd.isnull(genotypes['Def-S5']):
                    kdr_S_origins = f'{kdr_S_origins},(S4S5)_hom'
                elif genotypes['Def-S5'] == 'CC':
                    kdr_S_origins = f'{kdr_S_origins},S5_hom'
                elif genotypes['Def-S5'] == 'AC':
                    kdr_S_origins = f'{kdr_S_origins},S5_het,S4_het'
                elif genotypes['Def-S5'] == 'AA':
                    kdr_S_origins = f'{kdr_S_origins},S4_hom'
            elif genotypes['Def-S4S5-2'] == 'GT':
                if pd.isnull(genotypes['Def-S5']):
                    kdr_S_origins = f'{kdr_S_origins},(S4S5)_het'
                elif genotypes['Def-S5'] == 'CC':
                    kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S4S5, but homozygote for S5.'
                elif genotypes['Def-S5'] == 'AC':
                    kdr_S_origins = f'{kdr_S_origins},S5_het'
                elif genotypes['Def-S5'] == 'AA':
                    kdr_S_origins = f'{kdr_S_origins},S4_het'
        else :
            if pd.isnull(genotypes['Def-S4S5']):
                kdr_S_origins = f'{kdr_S_origins},S4S5?'
            elif genotypes['Def-S4S5'] == 'CC':
                if pd.isnull(genotypes['Def-S5']):
                    kdr_S_origins = f'{kdr_S_origins},(S4S5)_hom'
                elif genotypes['Def-S5'] == 'CC':
                    kdr_S_origins = f'{kdr_S_origins},S5_hom'
                elif genotypes['Def-S5'] == 'AC':
                    kdr_S_origins = f'{kdr_S_origins},S4_het,S5_het'
                elif genotypes['Def-S5'] == 'AA':
                    kdr_S_origins = f'{kdr_S_origins},S4_hom'
            elif genotypes['Def-S4S5'] == 'CT':
                if pd.isnull(genotypes['Def-S5']):
                    kdr_S_origins = f'{kdr_S_origins},(S4S5)_het'
                elif genotypes['Def-S5'] == 'CC':
                    kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S4S5, but homozygote for S5.'
                elif genotypes['Def-S5'] == 'AC':
                    kdr_S_origins = f'{kdr_S_origins},S5_het'
                elif genotypes['Def-S5'] == 'AA':
                    kdr_S_origins = f'{kdr_S_origins},S4_het'
    # for S heterozygotes
    elif kdr_S_origins == 'S:het':
        if pd.isnull(genotypes['Def-S1-3']):
            kdr_S_origins = f'{kdr_S_origins},S1?'
        elif genotypes['Def-S1-3'] == 'CC':
            kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S kdr, but homozygote for S1.'
        elif genotypes['Def-S1-3'] == 'CT':
            kdr_S_origins = f'{kdr_S_origins},S1_het'
        #
        if pd.isnull(genotypes['Def-S2S4']):
            kdr_S_origins = f'{kdr_S_origins},S2S4?'
        elif genotypes['Def-S2S4'] == 'TT':
            kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S kdr, but homozygote for S2S4.'
        elif genotypes['Def-S2S4'] == 'CT':
            if pd.isnull(genotypes['Def-S2-4']):
                kdr_S_origins = f'{kdr_S_origins},(S2S4)_het'
            elif genotypes['Def-S2-4'] == 'AA':
                ksr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S kdr and S2S4, but homozygote for S2.'
            elif genotypes['Def-S2-4'] == 'AT':
                kdr_S_origins = f'{kdr_S_origins},S2_het'
            elif genotypes['Def-S2-4'] == 'TT':
                kdr_S_origins = f'{kdr_S_origins},S4_het'
        #
        if pd.isnull(genotypes['Def-S3']):
            kdr_S_origins = f'{kdr_S_origins},S3?'
        elif genotypes['Def-S3'] == 'GG':
            kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S kdr, but homozygote for S3.'
        elif genotypes['Def-S3'] == 'GT':
            kdr_S_origins = f'{kdr_S_origins},S3_het'
        # 
        if alternate_S4S5:
            if pd.isnull(genotypes['Def-S4S5_2']):
                kdr_S_origins = f'{kdr_S_origins},S4S5?'
            elif genotypes['Def-S4S5-2'] == 'TT':
                kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S kdr, but homozygote for S4S5.'
            elif genotypes['Def-S4S5-2'] == 'GT':
                if pd.isnull(genotypes['Def-S5']):
                    kdr_S_origins = f'{kdr_S_origins},(S4S5)_het'
                elif genotypes['Def-S5'] == 'CC':
                    kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S kdr and S4S5, but homozygote for S5.'
                elif genotypes['Def-S5'] == 'AC':
                    kdr_S_origins = f'{kdr_S_origins},S5_het'
                elif genotypes['Def-S5'] == 'AA':
                    kdr_S_origins = f'{kdr_S_origins},S4_het'
        else :
            if pd.isnull(genotypes['Def-S4S5']):
                kdr_S_origins = f'{kdr_S_origins},S4S5?'
            elif genotypes['Def-S4S5'] == 'CC':
                kdr_S_origins = f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S kdr, but homozygote for S4S5.'
            elif genotypes['Def-S4S5'] == 'CT':
                if pd.isnull(genotypes['Def-S5']):
                    kdr_S_origins = f'{kdr_S_origins},(S4S5)_het'
                elif genotypes['Def-S5'] == 'CC':
                    return(f'Fail. Genotypes suggest that sample {sample_name} is heterozygote for S kdr and S4S5, but homozygote for S5.')
                elif genotypes['Def-S5'] == 'AC':
                    kdr_S_origins = f'{kdr_S_origins},S5_het'
                elif genotypes['Def-S5'] == 'AA':
                    kdr_S_origins = f'{kdr_S_origins},S4_het'
    if clean:
        return(_kdr_gen_cleanup(kdr_S_origins))
    else:
        return(kdr_S_origins)

def _402_kdr_origin_gen(genotypes, clean = True):
    if 'sample_name' in genotypes.index:
        sample_name = genotypes['sample_name']
    else:
        sample_name = genotypes.name
    # Check for the 995F mutations
    if pd.isnull(genotypes['kdr-402L']):
        kdr_402_origins = '402:unknown'
    elif genotypes['kdr-402L'] == 'GG':
        kdr_402_origins = '402:wt_hom'
    elif genotypes['kdr-402L'] == 'CG':
        kdr_402_origins = '402:het,402LC_het'
    elif genotypes['kdr-402L'] == 'GT':
        kdr_402_origins = '402:het,402LT_het'
    elif genotypes['kdr-402L'] == 'CT':
        kdr_402_origins = '402:hom,402LC_het,402LT_het'
    elif genotypes['kdr-402L'] == 'CC':
        kdr_402_origins = '402:hom,402LC_hom'
    elif genotypes['kdr-402L'] == 'TT':
        kdr_402_origins = '402:hom,402LT_hom'
    else:
        print(f'Unexpected kdr 402 genotype. {sample_name} {genotypes["kdr-402L"]}')
        kdr_402_origins = 'Fail. Unexpected kdr 402 genotype'
        
    if clean:
        return(_kdr_gen_cleanup(kdr_402_origins))
    else:
        return(kdr_402_origins)

# The initial output of the kdr_origin function can be a little messy, since it outputs all of the 
# information that it could or couldn't obtain. This function tidies it up a bit. 
def _kdr_gen_cleanup(kdr_origin_str):
    if re.search('Fail', kdr_origin_str):
        return('?,?')
    if re.search('wt', kdr_origin_str):
        return('wt,wt')
    kdr_type = re.findall('.*(?=:)', kdr_origin_str)[0]
    outcomes = kdr_origin_str.split(',')
    origins = np.unique(outcomes[1:])
    established_origins = [o for o in origins if not re.search('\?', o)]
    # Remove "_het" and "_hom" text
    established_origins = [re.sub('_het', '', o) for o in established_origins]
    # Now double up each _hom entry. this is cludgy, but coudn't find a more elegant way
    for i in range(len(established_origins)):
        o = established_origins[i]
        if re.search('_hom', o):
            o = re.sub('_hom', '', o)
            established_origins[i] = o
            established_origins.append(o)
    if re.search('hom', outcomes[0]):
        if len(established_origins) == 1:
            return(f'{kdr_type},{established_origins[0]}')
        elif len(established_origins) == 2:
            return(','.join(established_origins))
        else:
            return(f'{kdr_type},{kdr_type}')
    if re.search('het', outcomes[0]):
        if len(established_origins) == 1:
            return(f'wt,{established_origins[0]}')
        else:
            return(f'wt,{kdr_type}')
    else:
        return('?,?')


# Single function to call both the F and S origins for a given haplotype. This function determines
# from the look of the genotype table whether it represents genotypes or haplotypes, and calls 
# the appropriate function. 
def kdr_origin(genotypes, alternate_S4S5 = False, clean = True, include_402 = None):
    if 'sample_name' in genotypes.index:
        sample_name = genotypes['sample_name']
    else:
        sample_name = genotypes.name
    if include_402 == None:
        if 'kdr-402L' in genotypes.index:
            include_402 = True
        else:
            include_402 = False
    if include_402 == False:
        kdr_origins = pd.DataFrame({'kdr_F_origin': [_F_kdr_origin_gen(genotypes, clean)], 
                                    'kdr_S_origin': [_S_kdr_origin_gen(genotypes, clean, alternate_S4S5)]
                                    }, index = [sample_name]
        
        )
    else:
        kdr_origins = pd.DataFrame({'kdr_F_origin': [_F_kdr_origin_gen(genotypes, clean)], 
                                    'kdr_S_origin': [_S_kdr_origin_gen(genotypes, clean, alternate_S4S5)],
                                    'kdr_402_origin': [_402_kdr_origin_gen(genotypes, clean)]
                                    }, index = [sample_name]
        )
    return(kdr_origins)

# From a pair of kdr origin calls (obtained by running the kdr_origin function  for each of 
# F and S, followed by kdr_hap_cleanup), output a single call combining the F and S calls. 
def get_single_gen_call(x):  
    if 'kdr_402_origin' in x.index:
        return(_get_single_gen_call_with_402(x))
    else:
        return(_get_single_gen_call_no_402(x))
    
def _get_single_gen_call_no_402(x): 
    if 'sample_name' in x.index:
        sample_name = x['sample_name']
    else:
        sample_name = x.name
    joined_calls = np.array(x['kdr_F_origin'].split(',') + x['kdr_S_origin'].split(','))
    # There should be at least two 'wt' calls
    if np.sum(joined_calls == 'wt') < 2:
        print(f'Too many different mutant haplotype backgrounds in sample {sample_name}')
        return('?,?')
    # Otherwise, drop two wildtype calls
    else:
        which_drop = np.where(joined_calls == 'wt')[0][:2]
        return(','.join(np.delete(joined_calls, which_drop)))

def _get_single_gen_call_with_402(x): 
    if 'sample_name' in x.index:
        sample_name = x['sample_name']
    else:
        sample_name = x.name
    joined_calls = np.array(x['kdr_F_origin'].split(',') + 
                            x['kdr_S_origin'].split(',') +
                            x['kdr_402_origin'].split(',')
    )
    # There should be at least four 'wt' calls
    if np.sum(np.isin(joined_calls,  ['wt', '?'])) < 4:
        print(f'Too many different mutant haplotype backgrounds in sample {sample_name}')
        return('?,?')
    # Otherwise, drop four wildtype calls
    else:
        which_drop = np.concatenate([
            np.where(joined_calls == 'wt')[0],
            np.where(joined_calls == '?')[0]
        ])[:4]
        return(','.join(np.delete(joined_calls, which_drop)))

### Load metadata

In [5]:
metadata = pd.read_csv(metadata_path, sep = '\t', index_col = 0)
metadata.taxon.fillna('UNKN', inplace = True)
metadata

,sampleID,taxon,location,country,plate,well_letter,well_number,latitude,longitude
0,GH_01,UNKN,Ghana,Ghana,3,A,1,NaN,NaN
1,GH_02,UNKN,Ghana,Ghana,3,A,2,NaN,NaN
3,GH_04,UNKN,Ghana,Ghana,3,A,4,NaN,NaN
4,GH_05,UNKN,Ghana,Ghana,3,A,5,NaN,NaN
5,GH_06,UNKN,Ghana,Ghana,3,A,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...
665,Siaya_Delta_Alive_128,gambiae,Siaya,Kenya,7,H,6,NaN,NaN
666,Siaya_Delta_Alive_129,gambiae,Siaya,Kenya,7,H,7,NaN,NaN
667,Siaya_Delta_Alive_130,gambiae,Siaya,Kenya,7,H,8,NaN,NaN
668,Siaya_Delta_Alive_131,gambiae,Siaya,Kenya,7,H,9,NaN,NaN


### Get the genotype calls

### Filter the SNPs to just the ones useful for kdr origin analysis

In [6]:
geno, pos, contig, samples, ref, alt = load_vcf(vcf_path, metadata=metadata)

which_snps = (contig == '2L') & np.isin(pos, hap_def['variant_pos']) 

snp_calls = geno[which_snps, :, :]
pos = pos[which_snps]
alt = alt[which_snps, :]
ref = ref[which_snps]

/home/sanj/.local/lib/python3.10/site-packages/allel/io/vcf_read.py:1732: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'
  warnings.warn('invalid INFO header: %r' % header)


### Convert genotype calls to nucleotides

In [7]:
# combine ref and alt into a single matrix, and add a column of '?' at the end, so that 
# any genotype call of -1 (missing) draws the '?' character'
snp_alleles = np.concatenate([np.reshape(ref, (len(ref), 1)), 
                              alt,
                              np.full((len(ref), 1), '?')], 
                             axis = 1)

# Convert numberic calls to nucleotides, and sort each pair of nucleotides alphabetically
# (so, eg, the genotype 'TA' becomes 'AT')
snp_genotypes_3d = snp_alleles[
    np.array(np.arange(snp_alleles.shape[0])).reshape(snp_alleles.shape[0], 1, 1), 
    snp_calls
]
snp_genotypes = np.apply_along_axis(lambda x: ''.join(np.sort(x)), 2, snp_genotypes_3d)

# Store results in data frame
hap_def.index = hap_def['variant_pos']
gen_df = pd.DataFrame(
    np.transpose(snp_genotypes), 
    index = samples,
    columns = hap_def.loc[pos, 'SNP name']
)
gen_df

SNP name,Def-S1-3,Def-F5-2,Def-S4S5-2,Def-F3,Def-S4S5,Def-S2S4,Def-S5,Def-F3F4-2,kdr-995S,kdr-995F,Def-F2,Def-S2-4,Def-S3,Def-F1
GH_01,TT,AA,GG,GG,TT,CC,AA,AA,TT,TT,GG,TT,TT,AA
GH_02,TT,AA,GG,GG,TT,CC,AA,AA,TT,TT,GG,TT,TT,AA
GH_04,TT,AA,GG,GG,TT,CC,AA,AA,TT,TT,GG,TT,TT,AA
GH_05,TT,AA,GG,GG,TT,CC,AA,AA,TT,TT,GG,TT,TT,AA
GH_06,TT,AA,GG,GG,TT,CC,AA,AA,TT,TT,GG,TT,TT,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Siaya_Delta_Alive_128,CC,AA,GG,GG,TT,CC,AA,AA,CT,AT,AA,TT,??,GG
Siaya_Delta_Alive_129,CT,AA,GG,GG,TT,CC,AA,AA,CT,AT,AG,TT,TT,GG
Siaya_Delta_Alive_130,CT,AA,GG,GG,TT,CC,AA,AA,CT,AT,AG,TT,TT,GG
Siaya_Delta_Alive_131,TT,AA,GG,GG,TT,CC,AA,AA,TT,TT,AA,TT,TT,GG


### Obtain kdr origin calls

In [8]:

kdr_origins = pd.concat([kdr_origin(gen_df.iloc[i]) for i in range(gen_df.shape[0])])
kdr_origins['kdr_origin'] = kdr_origins.apply(
    get_single_gen_call, axis = 1
)

kdr_origins

Unexpected kdr F genotype. GM_21 ??
Unexpected kdr S genotype. GM_21 ??
Unexpected kdr F genotype. GM_27 ??
Unexpected kdr S genotype. GM_27 ??
Unexpected kdr F genotype. GM_56 ??
Unexpected kdr S genotype. GM_56 ??
Unexpected kdr F genotype. Siaya_Delta_Alive_15 ??
Unexpected kdr S genotype. Siaya_Delta_Alive_15 ??
Too many different mutant haplotype backgrounds in sample GM_21
Too many different mutant haplotype backgrounds in sample GM_27
Too many different mutant haplotype backgrounds in sample GM_56
Too many different mutant haplotype backgrounds in sample Siaya_Delta_Dead_03
Too many different mutant haplotype backgrounds in sample Siaya_Delta_Dead_38
Too many different mutant haplotype backgrounds in sample Siaya_Delta_Dead_39
Too many different mutant haplotype backgrounds in sample Siaya_Delta_Dead_47
Too many different mutant haplotype backgrounds in sample Siaya_Delta_Dead_58
Too many different mutant haplotype backgrounds in sample Siaya_Delta_Dead_70
Too many different mut

,kdr_F_origin,kdr_S_origin,kdr_origin
GH_01,"F1,F1","wt,wt","F1,F1"
GH_02,"F1,F1","wt,wt","F1,F1"
GH_04,"F1,F1","wt,wt","F1,F1"
GH_05,"F1,F1","wt,wt","F1,F1"
GH_06,"F1,F1","wt,wt","F1,F1"
...,...,...,...
Siaya_Delta_Alive_128,"?,?","?,?","?,?"
Siaya_Delta_Alive_129,"wt,F2","wt,S1","F2,S1"
Siaya_Delta_Alive_130,"wt,F2","wt,S1","F2,S1"
Siaya_Delta_Alive_131,"F2,F2","wt,wt","F2,F2"


### Merge kdr origins with metadata and write to file. 

In [14]:
kdr_origins_df = pd.merge(kdr_origins, metadata.set_index("sampleID"), left_index = True, right_index = True)
kdr_origins_df.to_csv(f'{wkdir}/results/kdr-origins/kdr_origins.csv', sep = '\t')
kdr_origins_df

,kdr_F_origin,kdr_S_origin,kdr_origin,taxon,location,country,plate,well_letter,well_number,latitude,longitude
GH_01,"F1,F1","wt,wt","F1,F1",UNKN,Ghana,Ghana,3,A,1,NaN,NaN
GH_02,"F1,F1","wt,wt","F1,F1",UNKN,Ghana,Ghana,3,A,2,NaN,NaN
GH_04,"F1,F1","wt,wt","F1,F1",UNKN,Ghana,Ghana,3,A,4,NaN,NaN
GH_05,"F1,F1","wt,wt","F1,F1",UNKN,Ghana,Ghana,3,A,5,NaN,NaN
GH_06,"F1,F1","wt,wt","F1,F1",UNKN,Ghana,Ghana,3,A,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Siaya_Delta_Alive_128,"?,?","?,?","?,?",gambiae,Siaya,Kenya,7,H,6,NaN,NaN
Siaya_Delta_Alive_129,"wt,F2","wt,S1","F2,S1",gambiae,Siaya,Kenya,7,H,7,NaN,NaN
Siaya_Delta_Alive_130,"wt,F2","wt,S1","F2,S1",gambiae,Siaya,Kenya,7,H,8,NaN,NaN
Siaya_Delta_Alive_131,"F2,F2","wt,wt","F2,F2",gambiae,Siaya,Kenya,7,H,9,NaN,NaN


### Create a table where each row is a haplotype instead of a genotype (although these are genotype-based calls, so order within each sample will be random, so they can be used for, say, mapping, but not haplotype clustering). Write this table to file.

In [16]:

kdr_genhap_origins_df = pd.DataFrame({'kdr_origin': ','.join(list(kdr_origins['kdr_origin'])).split(',')},
                                     index = np.repeat(kdr_origins.index, 2)
)
kdr_genhap_origins_df = pd.merge(kdr_genhap_origins_df, metadata.set_index("sampleID"), left_index=True, right_index=True)
kdr_genhap_origins_df.to_csv(f'{wkdir}/results/kdr-origins/kdr_genhap_origins.csv', sep = '\t')
kdr_genhap_origins_df

,kdr_origin,taxon,location,country,plate,well_letter,well_number,latitude,longitude
Calvin_01,wt,UNKN,UNKN,UNKN,14,A,1,NaN,NaN
Calvin_01,wt,UNKN,UNKN,UNKN,14,A,1,NaN,NaN
Calvin_02,wt,UNKN,UNKN,UNKN,14,A,2,NaN,NaN
Calvin_02,wt,UNKN,UNKN,UNKN,14,A,2,NaN,NaN
Calvin_03,wt,UNKN,UNKN,UNKN,14,A,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...
VK7_dead_42,F1,coluzzii,VK7,Burkina Faso,10,H,5,NaN,NaN
random_arabiensis,wt,arabiensis,random,random,7,C,12,NaN,NaN
random_arabiensis,wt,arabiensis,random,random,7,C,12,NaN,NaN
random_coluzzii,F1,coluzzii,random,random,7,D,12,NaN,NaN


### Summarise results by taxon and location

In [31]:
cols_keep = cohort_cols + ['kdr_origin']
# Count the number of occurances of each haplotypes in each population
# "values" could be any column that isn't specified elsewhere in the function. But it's 
# not allowed to be blank, so we had to pick one. 
pop_origin_counts = kdr_genhap_origins_df[cols_keep].pivot_table(columns='kdr_origin', 
                                                                    index=cohort_cols,
                                                                    aggfunc=len
                                                                   ).fillna(0).astype(int)

# A function to round a number up to n_signif significant figures
def signif(x, n_figs):
    power = 10 ** np.floor(np.log10(np.abs(x).clip(1e-200)))
    rounded = np.round(x / power, n_figs - 1) * power
    return rounded

# Calculate row totals of non-"?" columns
if '?' in pop_origin_counts.columns:
    pop_origin_counts = pop_origin_counts.drop('?', axis = 1)

row_totals = pop_origin_counts.sum(axis = 1)
# Calculate origin frequencies. We exclude the "?" calls for this
pop_origin_freqs = pop_origin_counts.div(row_totals, axis = 0)
# Round to 2 significant figures
pop_origin_freqs = signif(pop_origin_freqs, 2)
print('Counts of origins:')
display(pop_origin_counts)
print('\n\nFrequencies of known origins:')
display(pop_origin_freqs)

Counts of origins:


kdr_origin           F   F1   F2   S   S1  S3   wt
location taxon                                    
Gambia   UNKN        5   11    0  11    0   0  129
Ghana    UNKN        0  154    0   0    0   0    0
Sante    UNKN        0    0    0   1    0   0   11
Siaya    gambiae     3    0  279  17  103   2    6
UNKN     UNKN        0    0    0   0    0   0  188
VK7      coluzzii    1  171    0   0    0   0    0
ml       UNKN        1    0    1   5    5  14    0
random   arabiensis  0    0    0   0    0   0    2
         coluzzii    0    2    0   0    0   0    0



Frequencies of known origins:


kdr_origin                F     F1     F2      S    S1      S3     wt
location taxon                                                       
Gambia   UNKN        0.0320  0.071  0.000  0.071  0.00  0.0000  0.830
Ghana    UNKN        0.0000  1.000  0.000  0.000  0.00  0.0000  0.000
Sante    UNKN        0.0000  0.000  0.000  0.083  0.00  0.0000  0.920
Siaya    gambiae     0.0073  0.000  0.680  0.041  0.25  0.0049  0.015
UNKN     UNKN        0.0000  0.000  0.000  0.000  0.00  0.0000  1.000
VK7      coluzzii    0.0058  0.990  0.000  0.000  0.00  0.0000  0.000
ml       UNKN        0.0380  0.000  0.038  0.190  0.19  0.5400  0.000
random   arabiensis  0.0000  0.000  0.000  0.000  0.00  0.0000  1.000
         coluzzii    0.0000  1.000  0.000  0.000  0.00  0.0000  0.000

### That's all folks